- Keys in file: ['Day_1', 'Meta']
- Keys inside Meta: ['Age', 'Code', 'Height', 'No. Days', 'Sex', 'Weight']
- Keys inside Day_1: ['MVC', 'Trial_01', 'Trial_02', 'Trial_03', 'Trial_04', 'Trial_05', 'Trial_06', 'Trial_07', 'Trial_08', 'Trial_09', 'Trial_10', 'Trial_11', 'Trial_12', 'Trial_13', 'Trial_14', 'Trial_15', 'Trial_16', 'Trial_17', 'Trial_18', 'Trial_19', 'Trial_20', 'Trial_21', 'Trial_22', 'Trial_23', 'Trial_24', 'Trial_25', 'Trial_26', 'Trial_27', 'Trial_28', 'Trial_29', 'Trial_30', 'Trial_31', 'Trial_32', 'Trial_33', 'Trial_34', 'Trial_35', 'Trial_36']
- Keys inside Trial_01 : ['Acc_Left_Foot', 'Acc_Left_Shank', 'Acc_Left_Thigh', 'Acc_Pelvis', 'Acc_Right_Foot', 'Acc_Right_Shank', 'Acc_Right_Thigh', 'Acc_T8', 'Ang_L5S1', 'Ang_Left_Ankle', 'Ang_Left_Hip', 'Ang_Left_Knee', 'Ang_Right_Ankle', 'Ang_Right_Hip', 'Ang_Right_Knee', 'EMG_Left_BF', 'EMG_Left_GM', 'EMG_Left_Gmax', 'EMG_Left_Gmed', 'EMG_Left_RF', 'EMG_Left_ST', 'EMG_Left_TA', 'EMG_Left_VL', 'EMG_Right_BF', 'EMG_Right_GM', 'EMG_Right_Gmax', 'EMG_Right_Gmed', 'EMG_Right_RF', 'EMG_Right_ST', 'EMG_Right_TA', 'EMG_Right_VL', 'Gyr_Left_Foot', 'Gyr_Left_Shank', 'Gyr_Left_Thigh', 'Gyr_Pelvis', 'Gyr_Right_Foot', 'Gyr_Right_Shank', 'Gyr_Right_Thigh', 'Gyr_T8', 'Label', 'Markers', 'Time']

=> here, we extract only the EMG contents: ones that start with "EMG_ ..."


In [1]:
import h5py
import numpy as np
import pandas as pd
from sklearn.utils import resample
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('../Shared')
import processing, Model, Visualization
from sklearn.model_selection import train_test_split
import modules


def Run_feature_wise(F_name):
    base_path = 'D:/Data/Gait-EMG/Mypredict/'
    f = h5py.File(base_path+f"{F_name}", "r")

    key_lst = list(f['Day_1'].keys())

    Features = ['mav', 'var', 'zc', 'iemg', 'wl', 'wamp', 'mavs', 'rms', 'ssc', 'msq', 'v3', 'ld', 'dabs', 'mfl', 'mpr', 'mnf', 'psr', 'arc1', 'arc2', 'arc3', 'arc4', 'cc1', 'cc2', 'cc3', 'cca', 'dwtc1', 'dwtc2', 'dwtpc1', 'dwtpc2', 'dwtpc3']

    Acc_all = []
    for trials in key_lst[1:]:
        print("\n\n===================================", trials, "===================================\n")
        trial = f["Day_1"][trials]

        label, valid_idx = modules.label_extraction(trial)
        emg_keys = [k for k in trial.keys() if k.startswith("EMG_")]
        X_emg = modules.stack_signals(trial, emg_keys, valid_idx)

        balanced_idx = modules.Downsample_to_balance_class(label)
        X_emg_bal, y_bal = X_emg[balanced_idx], label[balanced_idx]
        print("EMG:", X_emg_bal.shape, "y:", y_bal.shape)
        #X_emg_bal, y_bal = random_downsample(X_emg_bal, y_bal, fraction=0.5)
        #print("After downsampling - EMG:", X_emg_bal.shape, "y:", y_bal.shape)

        window_size, step_size = 200, 10
        num_channels, num_features = X_emg.shape[1], 30
        X_tmp, y_tmp = processing.sliding_window_with_labels(X=X_emg_bal, y=y_bal,  window_size=window_size, step_size=step_size)  # (num_windows, win_len, ch)

        all_X, all_y = modules.get_X_y(X_tmp, y_tmp)
        all_y = modules.y_change_to_int(all_y)
        all_X = all_X.reshape(-1, num_channels, num_features, 1)

        Sub_acc = []
        for feature_idx in range(num_features):
            X_feature = all_X[:, :, feature_idx]   # shape: (samples, num_channels, 1, 1)
            X_feature = X_feature.reshape(-1, num_channels, 1)
            X_train, X_test, y_train, y_test = train_test_split(X_feature, all_y, test_size=0.2, random_state=42)

            model = Model.build_model_1D(input_shape=X_feature.shape[1:], num_classes=len(np.unique(all_y)))
            history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=512, verbose=0)

            print(f"\t {trials} - Feature {feature_idx} ({Features[feature_idx]}): "
                f"val acc => {np.max(history.history['val_accuracy'])*100:.2f}%")
            Sub_acc.append(np.max(history.history['val_accuracy'])*100)
        Acc_all.append(Sub_acc)

    pd.DataFrame(Acc_all, columns=Features).to_csv(f"./feature_wise_{F_name}.csv", index=False)

In [ ]:
Run_feature_wise(F_name="MP101.hdf5")



=================================== Trial_01 ===================================

EMG: (40500, 16) y: (40500,)
Unique labels: [0.  1.  2.  3.  5.  6.  7.  8.1 8.2 8.3 8.4 8.5 8.6 8.7 8.8]
New labels: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
	 Trial_01 - Feature 0 (mav): val acc => 36.31%
	 Trial_01 - Feature 1 (var): val acc => 22.43%
	 Trial_01 - Feature 2 (zc): val acc => 90.83%
	 Trial_01 - Feature 3 (iemg): val acc => 95.66%
	 Trial_01 - Feature 4 (wl): val acc => 98.14%
	 Trial_01 - Feature 5 (wamp): val acc => 55.27%


In [ ]:
Run_feature_wise(F_name="MP102.hdf5")

In [ ]:
Run_feature_wise(F_name="MP103.hdf5")

In [ ]:
Run_feature_wise(F_name="MP104.hdf5")

In [ ]:
Run_feature_wise(F_name="MP105.hdf5")

In [ ]:
Run_feature_wise(F_name="MP106.hdf5")

In [ ]:
Run_feature_wise(F_name="MP107.hdf5")

In [ ]:
Run_feature_wise(F_name="MP108.hdf5")

In [ ]:
Run_feature_wise(F_name="MP109.hdf5")

In [ ]:
Run_feature_wise(F_name="MP110.hdf5")